In [ ]:
from etl_lib.io import read_from_adls
from utils.env_utils import get_spark_session
from etl_lib.transformations import *
from etl_lib.steps import comupte_gold_transformation
import logging

In [ ]:
spark = get_spark_session()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
client_id = "..."
client_secret = "..."
tenant_id = "..."
storage_account = "stgetlprj01"

# Set up OAuth connection to ADLS Gen2
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net",
               f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [ ]:
path = f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/*.parquet"
df_silver = read_from_adls(path=path, spark=spark, file_format="parquet")

In [ ]:

path = f"abfss://gold@{storage_account}.dfs.core.windows.net/retail"
comupte_gold_transformation(df=df_silver, base_path=path, n=10)
